In [1]:
%load_ext autoreload
%autoreload 2
import sys
import csv
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT/data/action')

In [17]:
original = 'kookmin2_mmpose_train_s19-20' # 'kookmin_hrnet_modified_train_s01-2'
data = readpkl('./{}.pkl'.format(original))
data.keys()
print(data['split'].keys())
print(data['split']['train'])
print(data['split']['test'])
num_videos = len(data['annotations'])
print('number of videos : ', num_videos) # number of videos
print(data['annotations'][0].keys())
print('video name : ', data['annotations'][0]['frame_dir']) # video name
print('action label : ', data['annotations'][0]['label']) # action label
print('W, H : ', data['annotations'][0]['img_shape']) # W, H
print('number of frames : ', data['annotations'][0]['total_frames']) # number of frames
print('number of people : ', data['annotations'][0]['num_person_raw']) # number of people
print('keypoint : ', data['annotations'][0]['keypoint'].shape) # keypoint (num_person, total_frames, 17, 2)
print('confidence : ', data['annotations'][0]['keypoint_score'].shape) # confidence (num_person, total_frames, 17)

dict_keys(['train', 'test'])
['S20_burpee_4', 'S20_benchpress_1', 'S20_squat_5', 'S20_cat_2', 'S20_squat_2', 'S20_cat_5', 'S20_pushup_5', 'S20_pushup_3', 'S20_benchpress_4', 'S20_pushup_4', 'S20_benchpress_3', 'S20_burpee_2', 'S20_yoga_3', 'S20_yoga_2', 'S20_kneeup_3', 'S20_burpee_5', 'S20_burpee_1', 'S20_cat_4', 'S20_kneeup_5', 'S20_cat_1', 'S20_squat_4', 'S20_yoga_1', 'S20_squat_3', 'S20_kneeup_1', 'S20_burpee_3', 'S20_benchpress_5', 'S20_cat_3', 'S20_kneeup_4', 'S20_yoga_5', 'S20_pushup_1', 'S20_squat_1', 'S20_yoga_4', 'S20_pushup_2', 'S20_benchpress_2', 'S20_kneeup_2', 'S19_yoga_1', 'S19_yoga_2', 'S19_benchpress_3', 'S19_kneeup_5', 'S19_pushup_4', 'S19_kneeup_2', 'S19_pushup_1', 'S19_pushup_5', 'S19_yoga_5', 'S19_squat_5', 'S19_benchpress_1', 'S19_benchpress_4', 'S19_squat_4', 'S19_pushup_3', 'S19_burpee_3', 'S19_cat_1', 'S19_cat_2', 'S19_kneeup_4', 'S19_burpee_2', 'S19_yoga_3', 'S19_cat_3', 'S19_kneeup_1', 'S19_squat_2', 'S19_benchpress_2', 'S19_kneeup_3', 'S19_cat_4', 'S19_yoga_4

In [18]:
segment_length = 243
stride = 50
augment_train = True
augment_test = True

assert (augment_train or augment_test) == True, 'no augmentation'

modified = './'+original
if augment_train:
    modified += '_augment_{}_{}'.format(segment_length, stride)
if augment_test:
    modified += '_test_augment_{}_{}'.format(segment_length, stride)
modified += '.pkl'
print(modified)

./kookmin2_mmpose_train_s19-20_augment_243_50_test_augment_243_50.pkl


In [19]:
new_data = copy.deepcopy(data) # {'annotations':[], 'split': {'train': [], 'test': []}}

for i in range(num_videos):
    video_name = data['annotations'][i]['frame_dir']
    if (video_name in data['split']['train']):
        data_type = 'train'
    elif (video_name in data['split']['test']):
        data_type = 'test'
    if (data_type == 'train') and (not augment_train): continue # skip test videos
    if (data_type == 'test') and (not augment_test): continue # skip train videos
    label = data['annotations'][i]['label']
    W, H = data['annotations'][i]['img_shape']
    num_frames = data['annotations'][i]['total_frames']
    num_people = data['annotations'][i]['num_person_raw']
    keypoint = data['annotations'][i]['keypoint']
    confidence = data['annotations'][i]['keypoint_score']
    
    for j in range(0, num_frames, stride):
        if j + segment_length > num_frames:
            break
        #print(j)
        new_video_name = video_name+'_segment{}'.format(j)
        new_data['annotations'].append({'frame_dir': new_video_name, 'label': label, 'img_shape': data['annotations'][i]['img_shape'], 'total_frames': segment_length, 'num_person_raw': num_people, 'keypoint': keypoint[:, j:j+segment_length, :, :], 'keypoint_score': confidence[:, j:j+segment_length, :]})
        new_data['split'][data_type].append(new_video_name)
len(new_data['split']['train']), len(new_data['split']['test'])

(1705, 7405)

In [51]:
savepkl(new_data, modified)

In [14]:
data = readpkl(modified)
data['split']['train']

['S14_yoga_1',
 'S13_kneeup_3',
 'S14_benchpress_5',
 'S13_squat_3',
 'S14_kneeup_2',
 'S13_burpee_4',
 'S13_benchpress_3',
 'S13_burpee_1',
 'S14_squat_3',
 'S14_benchpress_3',
 'S13_burpee_2',
 'S14_cat_1',
 'S14_cat_5',
 'S14_burpee_4',
 'S14_cat_4',
 'S14_benchpress_4',
 'S13_yoga_2',
 'S13_yoga_1',
 'S14_squat_2',
 'S14_cat_2',
 'S13_pushup_2',
 'S13_squat_4',
 'S13_pushup_1',
 'S13_yoga_4',
 'S14_yoga_3',
 'S13_pushup_3',
 'S13_cat_4',
 'S14_pushup_3',
 'S13_squat_2',
 'S14_pushup_1',
 'S14_kneeup_3',
 'S13_yoga_5',
 'S13_cat_2',
 'S14_squat_1',
 'S14_burpee_1',
 'S13_yoga_3',
 'S13_squat_5',
 'S13_cat_3',
 'S14_burpee_3',
 'S13_benchpress_5',
 'S14_burpee_2',
 'S14_yoga_4',
 'S13_squat_1',
 'S13_pushup_4',
 'S14_pushup_2',
 'S13_cat_1',
 'S13_kneeup_2',
 'S14_yoga_2',
 'S13_burpee_3',
 'S14_benchpress_1',
 'S14_kneeup_5',
 'S13_kneeup_4',
 'S14_burpee_5',
 'S13_benchpress_1',
 'S13_benchpress_2',
 'S14_yoga_5',
 'S14_squat_5',
 'S14_pushup_4',
 'S13_benchpress_4',
 'S14_pushup_5

In [9]:
data = readpkl('kookmin2_mmpose_train_s19-20_augment_243_50_test_augment_243_50.pkl')
len(data['split']['train']), len(data['split']['test'])

(1705, 7405)